# Deliverable 2

SNumbers: u264332, u264443, u264202

Names: Levente Olivér Bódi, Riccardo Zamuner, Giada Izzo

## Previous deliverable code

This is the same code of the previous deliverable minus prints and commentary

In [21]:
import re
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from wordcloud import WordCloud

nltk.download('stopwords')
nltk.download('punkt_tab')


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/just_riccio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/just_riccio/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [22]:
def preprocess_text(text):
    """
    Preprocess a text by tokenizing, lowercasing, removing stop words, and stemming.
    """
    
    # Tokenize the text into words
    tokens = nltk.word_tokenize(text)

    # Convert to lowercase
    tokens = [token.lower() for token in tokens]

    # Remove punctuation
    tokens = [re.sub(r"[^\w\s]", "", token) for token in tokens]
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Remove punctuation
    tokens = [re.sub(r"[^\w\s]", "", token) for token in tokens]

    # Stem the tokens
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # Remove empty strings
    tokens = [token for token in tokens if token]

    return tokens

def clean_seller(text):
    """
    Clean the seller field by removing unwanted trailing phrases.
    """

    # Remove unwanted trailing phrases and everything after them
    remove_phrases = [
        "Seller changed",
        "(Not Enough Ratin",
        "(New Sell"
    ]
    for phrase in remove_phrases:
        idx = text.find(phrase)
        if idx != -1:
            text = text[:idx]
    return text.strip()

def preprocess_non_textual(document):
    """
    Preprocess non-textual fields in the document.
    """

    # Discount preprocessing: convert from string "xx% off" to integer xx
    # also taking into account documents without discount
    if isinstance(document["discount"], str) and "%" in document["discount"]:
        document["discount"] = int(document["discount"][:document["discount"].find("%")])
    else:
        document["discount"] = 0
        
    # Merge all values from product_details dictionary and preprocess
    if isinstance(document["product_details"], dict):
        details_text = " ".join(str(v) for v in document["product_details"].values())
    elif isinstance(document["product_details"], list):
        # If it's a list of dicts, merge all values from all dicts
        details_text = " ".join(str(v) for d in document["product_details"] if isinstance(d, dict) for v in d.values())
    else:
        details_text = str(document["product_details"])
    document["product_details"] = preprocess_text(details_text)

    # Convert actual_price and selling_price to integers (remove commas)
    # If actual_price is NaN, set it to discounted selling_price
    for price_field in ["actual_price", "selling_price"]:
        if isinstance(document[price_field], str):
            price_str = document[price_field].replace(",", "")
            price_val = price_str.split(".")[0]
            document[price_field] = int(price_val) if price_val.isdigit() else 0

    # If actual_price is missing or zero, set it to discounted selling_price
    if ("actual_price" not in document or document["actual_price"] == 0) and "selling_price" in document:
        document["actual_price"] = int(int(document["selling_price"])*document["discount"]/100)

    # Convert average_rating to float, set to NaN if missing or empty
    if "average_rating" in document and str(document["average_rating"]).strip() != "":
        try:
            document["average_rating"] = float(document["average_rating"])
        except ValueError:
            document["average_rating"] = float("nan")
    else:
        document["average_rating"] = float("nan")

    return document

def preprocess_document(document):
    """
    Join all preprocessing steps for a document.
    """

    document["description"] = preprocess_text(document["description"])
    document["title"] = preprocess_text(document["title"])
    document["seller"] = clean_seller(document["seller"])
    document["brand"] = document["brand"].lower().split()

    document = preprocess_non_textual(document)

    return document


In [23]:
# MODIFY THIS PATH AS NEEDED
file_path = "../../data/fashion_products_dataset.json"

with open(file_path, "r") as f:
    data = json.load(f)
    df = pd.DataFrame(data)

In [24]:
def impute_actual_price(row):
    # if actual_price is empty, try to compute it:
    # either from selling_price and discount, or just use selling_price
    if row['actual_price'] == '':
        # Convert selling_price and discount to float if not empty
        if row['selling_price'] != '' and row['discount'] != '':
            selling_price = float(str(row['selling_price']).replace(',', ''))
            discount = float(str(row['discount']).replace('%', '').replace('off', '').strip())
            return selling_price * (1 - discount / 100)
        elif row['selling_price'] != '':
            return float(str(row['selling_price']).replace(',', ''))
    return row['actual_price']

df['discount'] = df['discount'].replace('', '0')
df['actual_price'] = df.apply(impute_actual_price, axis=1)

In [25]:
# Drop the remaining products without price
df = df[(df['actual_price'] != '') & (df['selling_price'] != '')]

In [26]:
# Replace empty brand names with 'no brand'
df.loc[df['brand'] == '', 'brand'] = 'no brand'

In [27]:
df = df.apply(preprocess_document, axis=1)

# Deliverable 2

## Part 1

### Build index

In [28]:
from collections import defaultdict

def normalize_cat_token(val):
    if pd.isna(val) or str(val).strip() == "":
        return []
    # split common multi-value strings; keep a single value as 1-item list
    parts = re.split(r"[\/,;|]", str(val))
    return [re.sub(r"\s+", "_", p.strip().lower()) for p in parts if p.strip()]

In [29]:
print(df.columns)

Index(['_id', 'actual_price', 'average_rating', 'brand', 'category',
       'crawled_at', 'description', 'discount', 'images', 'out_of_stock',
       'pid', 'product_details', 'seller', 'selling_price', 'sub_category',
       'title', 'url'],
      dtype='object')


In [30]:
def build_inverted_index_df(df: pd.DataFrame,id_col: str | None = None,text_cols: list[str] = ("title", "description", "product_details"),min_df: int = 1,store_positions: bool = False):
    """
    df: preprocessed dataframe (title/description/product_details are token lists).
    id_col: column holding unique ids; if None, uses df.index (as str).
    text_cols: columns with *token lists* (already stemmed, stopwords removed).
    min_df: drop terms that appear in < min_df documents.
    store_positions: if True, also keep term positions for phrase/proximity queries.
    """
    # assign doc ID's
    doc_ids = df[id_col].astype(str).tolist() if id_col else df.index.astype(str).tolist()

    per_doc_terms = []
    per_doc_sequence = []

    # gather tokens for each row
    for _, row in df.iterrows():
        tokens = []

        # text cols are already tokenized lists after preprocess_document(), we just make it robust if something slipped through
        for c in text_cols:
            if c in df.columns:
                vals = row[c]
                if isinstance(vals, (list, tuple)):
                    tokens.extend([str(t).lower() for t in vals if str(t).strip()])
                elif pd.notna(vals):
                    # if something slipped through as string, tokenize lightly:
                    tokens.extend(re.findall(r"[A-Za-z0-9]+", str(vals).lower()))


        # ensure we have a sequence for positions and a set for boolean presence
        if store_positions:
            per_doc_sequence.append(tokens[:])
        per_doc_terms.append(set(tokens))

    # build postings (term -> list[doc_id]) and df counts
    postings_tmp = defaultdict(list)
    df_count = defaultdict(int)

    for d_i, terms in enumerate(per_doc_terms):
        did = doc_ids[d_i]
        for term in terms:
            postings_tmp[term].append(did)
            df_count[term] += 1

    # min_df filter + sort postings
    postings_tmp = {t: sorted(dids) for t, dids in postings_tmp.items() if df_count[t] >= min_df}

    # vocab
    vocab = {term: tid for tid, term in enumerate(sorted(postings_tmp.keys()))}
    id2term = {tid: term for term, tid in vocab.items()}

    # final inverted index (term_id -> [doc_ids])
    inv_index = {vocab[t]: dids for t, dids in postings_tmp.items()}

    # positional index
    positional = None
    if store_positions:
        positional = {tid: defaultdict(list) for tid in inv_index.keys()}
        for d_i, seq in enumerate(per_doc_sequence):
            did = doc_ids[d_i]
            for pos, tok in enumerate(seq):
                if tok in vocab:
                    tid = vocab[tok]
                    positional[tid][did].append(pos)
        # convert inner dicts to normal dicts
        positional = {tid: dict(dmap) for tid, dmap in positional.items()}

    return {
        "vocab": vocab,            # term -> term_id
        "id2term": id2term,        # term_id -> term
        "postings": inv_index,     # term_id -> [doc_id, ...] (sorted)
        "doc_ids": doc_ids,        # all doc ids, as strings
        "positional": positional   # optional: term_id -> {doc_id: [positions]}
    }

index_obj = build_inverted_index_df(
    df,
    id_col=None,
    text_cols=df.columns,
    min_df=1,
    store_positions=False
)

In [31]:
# Quick lookups
def docs_for_term(term: str):
    """Return document IDs for a raw term or categorical token (e.g., 'brand:nike')."""
    tid = index_obj["vocab"].get(term)
    return index_obj["postings"].get(tid, []) if tid is not None else []

def doc_positions_for_term(term: str, doc_id: str):
    """Return positions of term in a specific document."""
    tid = index_obj["vocab"].get(term)
    if tid is None:
        return []
    return index_obj["positional"].get(tid, {}).get(doc_id, [])

def and_query(terms: list[str]):
    """Boolean AND over terms."""
    sets = [set(docs_for_term(t)) for t in terms]
    return sorted(set.intersection(*sets)) if sets else []

def or_query(terms: list[str]):
    """Boolean OR over terms."""
    s = set()
    for t in terms:
        s.update(docs_for_term(t))
    return sorted(s)


# Plain term from text columns 
# Print only the first results to not flood the output
print("docs for 'cotton':", docs_for_term("cotton")[:50])
print("docs for adidas:", docs_for_term("adidas")[:50])
print("docs for category tshirts:", docs_for_term("tshirts")[:50])
print("cotton AND adidas:", and_query(["cotton", "adidas"])[:50])
print("hoodie OR sweatshirt:", or_query(["hoodie", "sweatshirt"])[:50])
print("docs for 50", docs_for_term("50")[:50])

docs for 'cotton': ['0', '1', '10', '1000', '1001', '1002', '1003', '1004', '1005', '1006', '1007', '1008', '1009', '10099', '1010', '1011', '1012', '1013', '1014', '1015', '1016', '10221', '10222', '10223', '10227', '10230', '10234', '10237', '10238', '10239', '10241', '10242', '10243', '10245', '10246', '10248', '10249', '1025', '10250', '10252', '10253', '10256', '10257', '1026', '10264', '10266', '10268', '1027', '10276', '10278']
docs for adidas: ['250', '251', '252', '253', '254', '255', '256', '257', '258', '259', '260', '261', '262', '263', '264', '265', '266', '267', '268', '269', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '280', '281', '282', '283', '284', '285', '286', '287', '288', '289', '290', '291', '292', '293', '294', '295', '296', '297', '298', '299']
docs for category tshirts: ['13029', '13030', '13031', '13032', '13033', '13034', '13035', '13036', '13037', '13038', '13039', '13040', '13041', '13042', '13043', '13044', '13045', '13046', '13

### Example queries

In [32]:
from collections import Counter

all_terms = [term for tokens in df["description"] for term in tokens]
term_freq = Counter(all_terms)
print(term_freq.most_common(20))


test_queries = {
    "Q1": ["cotton", "tshirt", "50", "100", "men", "blue"],
    "Q2": ["adidas", "red"],
    "Q3": ["denim", "jean", "skinny"],
    "Q4": ["dress", "red"],
    "Q5": ["leather", "jacket"]
}

for qid, terms in test_queries.items():
    result_docs = and_query(terms)
    print(f"{qid} -> {len(result_docs)} results: {result_docs[:10]}")


[('tshirt', 12385), ('cotton', 9944), ('wear', 9250), ('comfort', 8654), ('shirt', 7493), ('look', 6173), ('casual', 5926), ('fit', 5600), ('made', 5591), ('fabric', 5438), ('print', 4870), ('women', 4416), ('design', 4237), ('sleev', 4009), ('qualiti', 4002), ('men', 3731), ('day', 3463), ('wash', 3453), ('style', 3241), ('make', 3233)]
Q1 -> 30 results: ['11237', '11276', '11278', '19230', '19252', '19427', '19461', '20373', '23364', '23524']
Q2 -> 3 results: ['254', '268', '285']
Q3 -> 68 results: ['11502', '11513', '11527', '11580', '11596', '11628', '11629', '11637', '11648', '11657']
Q4 -> 62 results: ['120', '122', '126', '134', '14264', '149', '17411', '23788', '26308', '26884']
Q5 -> 56 results: ['1018', '1029', '1030', '10344', '1040', '1041', '1126', '13175', '14094', '14170']


### Build TF-IDF Ranking

In [33]:
def calculate_tf(word, document):
    """
    Calculate term frequency for a word in a document.
    TF = Number of times term t appears in a document
    """
    return document.count(word)    
    

def calculate_idf(word, all_documents):
    """
    Calculate inverse document frequency for a word.
    IDF = log(Total number of documents / Number of documents containing term t)
    """
    num_documents_with_term = len(docs_for_term(word))
    if num_documents_with_term == 0:
        return 0
    return np.log(len(all_documents) / num_documents_with_term)

def cosine_similarity(vec1, vec2):
    """
    Calculate cosine similarity between two vectors.
    """
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0
    return dot_product / (norm_vec1 * norm_vec2)


In [34]:
def rank_documents(query, documents, k):
    """
    Rank documents based on TF-IDF scores for the given query.
    Return the top k documents.
    """
    all_documents = [doc["description"] + doc["title"] + doc["brand"] for index, doc in documents.iterrows()]
    scores = []

    term_idfs = {term: calculate_idf(term, all_documents) for term in query}
    query_vector = np.array([calculate_tf(term, query) * term_idfs[term] for term in query])

    for index, doc in documents.iterrows():
        doc_vec = []
        doc_text = doc["description"] + doc["title"] + doc["brand"]
        for term in query:
            tf = calculate_tf(term, doc_text)
            if tf > 0:
                # used the formula tf = 1 + log_10(count)
                doc_vec.append((1 + np.log(tf)) * term_idfs[term])
            else:
                doc_vec.append(0)
        scores.append((doc, cosine_similarity(query_vector, np.array(doc_vec))))

    # Sort documents by score in descending order
    ranked_docs = sorted(scores, key=lambda x: x[1], reverse=True)

    return ranked_docs[:k]

In [35]:
out = rank_documents(test_queries["Q1"], df, k=5)
display(out)

[(_id                             dea0b151-727d-51d7-b902-475dc3db64c7
  actual_price                                                   499.0
  average_rating                                                   3.8
  brand                                              [naaz, collectio]
  category                                    Clothing and Accessories
  crawled_at                                             1612999518000
  description        [naaz, collect, bring, limit, edit, 100, cotto...
  discount                                                          40
  images             [https://rukminim1.flixcart.com/image/128/128/...
  out_of_stock                                                   False
  pid                                                 TSHFVJZ3RMKUZSJC
  product_details    [round, neck, full, sleev, regular, cotton, je...
  seller                                             Anishaonlinestore
  selling_price                                                    299
  sub_

## Part 2

### Evaluation metrics

In [36]:
def precision_at_k(retrieved_docs, relevant_docs, k):
    """
    Calculate Precision@k.
    Precision@k = (Number of relevant documents retrieved in top k) / k
    """
    retrieved_at_k = retrieved_docs[:k]
    relevant_retrieved = sum(1 for doc in retrieved_at_k if doc["pid"] in relevant_docs)
    return relevant_retrieved / k if k > 0 else 0

def recall_at_k(retrieved_docs, relevant_docs, k):
    """
    Calculate Recall@k.
    Recall@k = (Number of relevant documents retrieved in top k) / (Total number of relevant documents)
    """
    retrieved_at_k = retrieved_docs[:k]
    relevant_retrieved = sum(1 for doc in retrieved_at_k if doc["pid"] in relevant_docs)
    total_relevant = len(relevant_docs)
    return relevant_retrieved / total_relevant if total_relevant > 0 else 0

def average_precision_at_k(retrieved_docs, relevant_docs, k):
    """
    Calculate Average Precision@k.
    AP@k = Average of Precision@i for each relevant document retrieved in top k
    """
    retrieved_at_k = retrieved_docs[:k]
    relevant_retrieved = 0
    precision_sum = 0

    for i, doc in enumerate(retrieved_at_k, start=1):
        if doc["pid"] in relevant_docs:
            relevant_retrieved += 1
            precision_sum += relevant_retrieved / i

    return precision_sum / relevant_retrieved if relevant_retrieved > 0 else 0

def f1_score(precision, recall):
    """
    Calculate F1 Score.
    F1 = 2 * (Precision * Recall) / (Precision + Recall)
    """
    if precision + recall == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)

def f1_score_at_k(retrieved_docs, relevant_docs, k):
    """
    Calculate F1 Score at k.
    """
    precision = precision_at_k(retrieved_docs, relevant_docs, k)
    recall = recall_at_k(retrieved_docs, relevant_docs, k)
    return f1_score(precision, recall)

def mean_average_precision(retrieved_docs_list, relevant_docs_list, k):
    """
    Calculate Mean Average Precision (MAP) at k.
    MAP = Mean of Average Precision@k over all queries
    """
    ap_sum = 0
    num_queries = len(retrieved_docs_list)

    for retrieved_docs, relevant_docs in zip(retrieved_docs_list, relevant_docs_list):
        ap_sum += average_precision_at_k(retrieved_docs, relevant_docs, k)

    return ap_sum / num_queries if num_queries > 0 else 0

def reciprocal_rank(retrieved_docs, relevant_docs):
    """
    Calculate Reciprocal Rank (RR).
    RR = 1 / Rank of the first relevant document
    """
    rank = 0
    for i, doc in enumerate(retrieved_docs):
        if doc["pid"] in relevant_docs:
            rank = i + 1
            break
    return 1 / rank if rank > 0 else 0

def mean_reciprocal_rank(retrieved_docs_list, relevant_docs_list):
    """
    Calculate Mean Reciprocal Rank (MRR).
    MRR = Mean of Reciprocal Ranks over all queries
    """
    rr_sum = 0
    num_queries = len(retrieved_docs_list)

    for retrieved_docs, relevant_docs in zip(retrieved_docs_list, relevant_docs_list):
        rr_sum += reciprocal_rank(retrieved_docs, relevant_docs)

    return rr_sum / num_queries if num_queries > 0 else 0

def dcg_at_k(retrieved_docs, relevant_docs, k):
    """
    Calculate Discounted Cumulative Gain (DCG) at k.
    DCG@k = Sum of (relevance of document at rank i) / log2(i + 1) for i in 1 to k
    """
    dcg = 0
    for i in range(min(k, len(retrieved_docs))):
        doc = retrieved_docs[i]
        if doc["pid"] in relevant_docs:
            relevance = 1  # we only have binary relevance
        else:
            relevance = 0
        dcg += relevance / np.log2(i + 2)  # i + 2 because i starts from 0
    return dcg

def ndcg_at_k(retrieved_docs, relevant_docs, k):
    """
    Calculate Normalized Discounted Cumulative Gain (NDCG) at k.
    NDCG@k = DCG@k / IDCG@k
    """
    dcg = dcg_at_k(retrieved_docs, relevant_docs, k)
    
    ideal_retrieved_docs = [{"pid": pid} for pid in relevant_docs]
    idcg = dcg_at_k(ideal_retrieved_docs, relevant_docs, k)
    
    return dcg / idcg if idcg > 0 else 0

In [37]:
# CHANGE THIS PATH AS NEEDED
relevant_docs_path = "../../data/validation_labels.csv"
relevant_df = pd.read_csv(relevant_docs_path)
relevant_q1 = relevant_df[relevant_df['query_id'] == 1]['pid'].tolist()
relevant_q2 = relevant_df[relevant_df['query_id'] == 2]['pid'].tolist()

example_q1 = "women full sleeve sweatshirt cotton".split()
example_q2 = "men slim jeans blue".split()

out1 = rank_documents(example_q1, df, k=15)
out2 = rank_documents(example_q2, df, k=15)

In [38]:
# Show only the first 5 results for brevity
display(out1[:5])
display(out2[:5])

[(_id                             b7d17523-6ce4-51fe-b56f-5c61e3c122c7
  actual_price                                                  2599.0
  average_rating                                                   4.0
  brand                                                         [voxa]
  category                                    Clothing and Accessories
  crawled_at                                             1612990563000
  description        [cotton, fleec, blend, slim, fit, two, front, ...
  discount                                                          76
  images             [https://rukminim1.flixcart.com/image/128/128/...
  out_of_stock                                                   False
  pid                                                 SWSFN8YZ33MHKQVF
  product_details    [maroon, cotton, fleec, blend, solid, rib, col...
  seller                                               LAFANTAR E-TAIL
  selling_price                                                    598
  sub_

[(_id                             db53a2e8-3b7d-59fb-9aa9-f8c1b4492e7b
  actual_price                                                  3299.0
  average_rating                                                   4.3
  brand                                                    [no, brand]
  category                                    Clothing and Accessories
  crawled_at                                             1612987934000
  description        [wear, eleg, man, tie, effect, way, make, look...
  discount                                                          72
  images             [https://rukminim1.flixcart.com/image/128/128/...
  out_of_stock                                                   False
  pid                                                 CTPFVZT7EFZWVRUP
  product_details    [formalcasu, print, polyest, neckti, set, pock...
  seller                                               NextEdgeRetails
  selling_price                                                    899
  sub_

In [39]:
# remove score as it is not needed for evaluation functions
retrieved1 = [doc for doc, score in out1]
retrieved2 = [doc for doc, score in out2]

def show_top_k(out, k=5):
    return [(doc['pid'], float(score)) for doc, score in out[:k]]


# ks to evaluate, we put only 15 to not clutter the output
ks = [15]

print('\nEvaluation for Query 1:')
for k in ks:
    p = precision_at_k(retrieved1, relevant_q1, k)
    r = recall_at_k(retrieved1, relevant_q1, k)
    f1 = f1_score_at_k(retrieved1, relevant_q1, k)
    ap = average_precision_at_k(retrieved1, relevant_q1, k)
    ndcg = ndcg_at_k(retrieved1, relevant_q1, k)
    print(f'  k={k}: Precision@k={p:.3f}, Recall@k={r:.3f}, F1@k={f1:.3f}, AP@k={ap:.3f}, NDCG@k={ndcg:.3f}')

print('\nEvaluation for Query 2:')
for k in ks:
    p = precision_at_k(retrieved2, relevant_q2, k)
    r = recall_at_k(retrieved2, relevant_q2, k)
    f1 = f1_score_at_k(retrieved2, relevant_q2, k)
    ap = average_precision_at_k(retrieved2, relevant_q2, k)
    ndcg = ndcg_at_k(retrieved2, relevant_q2, k)
    print(f'  k={k}: Precision@k={p:.3f}, Recall@k={r:.3f}, F1@k={f1:.3f}, AP@k={ap:.3f}, NDCG@k={ndcg:.3f}')

# MAP@k and MRR across the two queries
for k in [15]:
    map_k = mean_average_precision([retrieved1, retrieved2], [relevant_q1, relevant_q2], k)
    mrr_k = mean_reciprocal_rank([retrieved1, retrieved2], [relevant_q1, relevant_q2])
    print(f'\nMAP@{k}: {map_k:.3f}, MRR = {mrr_k:.3f}')


Evaluation for Query 1:
  k=15: Precision@k=0.067, Recall@k=0.050, F1@k=0.057, AP@k=0.500, NDCG@k=0.108

Evaluation for Query 2:
  k=15: Precision@k=0.067, Recall@k=0.050, F1@k=0.057, AP@k=0.083, NDCG@k=0.046

MAP@15: 0.292, MRR = 0.292


### Proposed queries evaluation

In [40]:
def get_pid_from_id(doc_id: str):
    return df.iloc[int(doc_id)]['pid']

# choice to automatically assign relevance (instead of going through some documents one by one):
# -retrieve all documents which contain all words in the query (like previous task)
# -if the number of retrieved documents is <MIN_AMOUNT, keep them all
# -otherwise keep a small percentage of the remaining documents after MIN_AMOUNT

MIN_AMOUNT = 30
PERCENTAGE = 0.20
relevant_docs = []

for qid, terms in test_queries.items():
    result_docs = and_query(terms)
    docs_to_retrieve = len(result_docs) if len(result_docs) < MIN_AMOUNT else MIN_AMOUNT + int(PERCENTAGE * (len(result_docs) - MIN_AMOUNT))
    result_docs = [get_pid_from_id(doc_id) for doc_id in result_docs[:docs_to_retrieve]]
    relevant_docs.append(result_docs)

retrieved_docs_list = [rank_documents(query, df, k=10) for query in test_queries.values()]
# remove the score from the retrieved docs as it is not needed for evaluation
retrieved_docs_list = [[doc for doc, score in out] for out in retrieved_docs_list]
ks = [5, 10]

for i, query in enumerate(test_queries.values()):
    retrieved = retrieved_docs_list[i]
    print('\nEvaluation for query: ' + ' '.join(query))
    for k in ks:
        p = precision_at_k(retrieved, relevant_docs[i], k)
        r = recall_at_k(retrieved, relevant_docs[i], k)
        f1 = f1_score_at_k(retrieved, relevant_docs[i], k)
        ap = average_precision_at_k(retrieved, relevant_docs[i], k)
        ndcg = ndcg_at_k(retrieved, relevant_docs[i], k)
        print(f'  k={k}: Precision@k={p:.3f}, Recall@k={r:.3f}, F1@k={f1:.3f}, AP@k={ap:.3f}, NDCG@k={ndcg:.3f}')

for k in ks:
    map_k = mean_average_precision(retrieved_docs_list, relevant_docs, k)
    mrr_k = mean_reciprocal_rank(retrieved_docs_list, relevant_docs)
    print(f'\nMAP@{k}: {map_k:.3f}, MRR = {mrr_k:.3f}')



Evaluation for query: cotton tshirt 50 100 men blue
  k=5: Precision@k=0.000, Recall@k=0.000, F1@k=0.000, AP@k=0.000, NDCG@k=0.000
  k=10: Precision@k=0.000, Recall@k=0.000, F1@k=0.000, AP@k=0.000, NDCG@k=0.000

Evaluation for query: adidas red
  k=5: Precision@k=0.600, Recall@k=1.000, F1@k=0.750, AP@k=1.000, NDCG@k=1.000
  k=10: Precision@k=0.300, Recall@k=1.000, F1@k=0.462, AP@k=1.000, NDCG@k=1.000

Evaluation for query: denim jean skinny
  k=5: Precision@k=0.000, Recall@k=0.000, F1@k=0.000, AP@k=0.000, NDCG@k=0.000
  k=10: Precision@k=0.000, Recall@k=0.000, F1@k=0.000, AP@k=0.000, NDCG@k=0.000

Evaluation for query: dress red
  k=5: Precision@k=0.000, Recall@k=0.000, F1@k=0.000, AP@k=0.000, NDCG@k=0.000
  k=10: Precision@k=0.300, Recall@k=0.083, F1@k=0.130, AP@k=0.216, NDCG@k=0.199

Evaluation for query: leather jacket
  k=5: Precision@k=1.000, Recall@k=0.143, F1@k=0.250, AP@k=1.000, NDCG@k=1.000
  k=10: Precision@k=1.000, Recall@k=0.286, F1@k=0.444, AP@k=1.000, NDCG@k=1.000

MAP@5

## Evaluation metrics explanation and analysis

### What each metric measures

- **Precision@k**: fraction of the top‑k results that are relevant. Uses only the top‑k set and binary relevance judgments.

- **Recall@k**: fraction of all relevant documents that appear in the top‑k. Uses top‑k and the full set of known relevant docs.

- **F1@k**: harmonic mean of Precision@k and Recall@k. Summarizes the balance between precision and recall at a given k.

- **Average Precision (AP@k)**: mean of precision values computed at ranks where relevant documents occur (truncated at k). Uses ordering information within top‑k and rewards early placement of relevant docs.

- **Mean Average Precision (MAP@k)**: mean of AP@k across queries. Used to assess the quality of the retrieval system.

- **Reciprocal Rank (RR) and Mean Reciprocal Rank (MRR)**: RR = 1 / rank of first relevant doc; MRR averages RR across queries. Uses only the first relevant hit.

- **DCG@k / NDCG@k**: discounted cumulative gain accounts for position (log discount); NDCG normalizes by ideal DCG so scores are comparable. Generally used with graded relevance (with binary relevance it simply accounts for discounted hits).

### Result interpretation

- High precision, low recall (small k): top results are clean but many relevant docs fall outside top‑k. To fix this we can increase recall via broader matching or query expansion, raise k, or tune retrieval. (This is the case for Q5)

- Low precision, high recall: system retrieves many relevant items but top positions contain non many non relevant documents. To fix this improve ranking, incorporate quality/popularity signals. May also be the fact there are not many relevant documents to begin with (like Q2, where one should actually use k = min(chosen_k, len(relevant_documents)))

- All metrics low: it can mean multiple things: preprocessing is not done well, tokenization may have mismatch, there could be indexing problems, or simply the retrieval model is too weak. (This is the problem for the other 3 queries, where the queries are pretty general and contain frequent terms)


### Retrieval problems and solution proposals

Right now the index and ranking focus almost entirely on token overlap in the text fields, but a lot of useful information in the product records is effectively ignored. Numeric fields like price, discount and rating are converted/cleaned but not used as ranked signals even though they can be powerful relevance cues for users (cheap vs premium, high rating, big discount), yet the current pipeline treats them as side data rather than features that should influence ranking. That means two items that match the query text equally well may be ordered arbitrarily even though one is clearly preferable by price or rating.

Another important issue is that all text fields are treated the same. Title, brand and description are concatenated or given equal treatment in scoring, so a match in a brand string carries the same weight as the same match in a long description. In practice users care much more about matches in short, high‑precision fields (title, brand) than in noisy long fields. Not giving fields different weights dilutes strong signals and reduces ranking accuracy.

Finally, the ranking ignores word order and phrases. The system is effectively bag‑of‑words: it counts tokens but doesn’t reward exact phrases or tokens that occur close together. Queries like “slim blue jeans” or “full sleeve sweatshirt” lose meaning when order and proximity aren’t considered; documents that contain the words scattered across different parts of the page can be ranked equally to documents that contain the exact phrase, which harms perceived relevance.

In short: useful numeric signals aren’t used as ranking features, field importance is flattened, and phrase/proximity information is lost. Some fixes proporals are the followings: treat numeric fields as features or filters, boost short/high‑precision fields (title, brand) when scoring, and enable positional/phrase handling (N-grams instead of unigrams).